# INF581 Lab5: Reinforcement Learning - TD Learning, QLearning and SARSA

<img src="https://raw.githubusercontent.com/jeremiedecock/polytechnique-inf581-2024-students/main/logo.jpg" style="float: left; width: 15%" />

[INF581-2024](https://moodle.polytechnique.fr/course/view.php?id=17108) Lab session #5

2019-2024 Jérémie Decock

[![Open in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeremiedecock/polytechnique-inf581-2024-students/blob/main/lab5_rl2_tdlearning_qlearning_sarsa.ipynb)

[![My Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jeremiedecock/polytechnique-inf581-2024-students/main?filepath=lab5_rl2_tdlearning_qlearning_sarsa.ipynb)

[![NbViewer](https://raw.githubusercontent.com/jupyter/design/main/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/jeremiedecock/polytechnique-inf581-2024-students/blob/main/lab5_rl2_tdlearning_qlearning_sarsa.ipynb)

[![Local](https://img.shields.io/badge/Local-Save%20As...-blue)](https://github.com/jeremiedecock/polytechnique-inf581-2024-students/raw/main/lab5_rl2_tdlearning_qlearning_sarsa.ipynb)

## Introduction

The purpose of this lab is to introduce some classic concepts used
in reinforcement learning: *Temporal Difference Learning* (*TD Learning*), *QLearning* and *SARSA*.

**Notice**: Here we assume that the reward only depends on the state: $r(\boldsymbol{s}) \equiv \mathcal{R}(\boldsymbol{s}, \boldsymbol{a}, \boldsymbol{s}')$.

## Python requirements

This notebook requires the following Python libraries: *Gymnasium*, NumPy, Pandas, Seaborn and Imageio.

### If you use Google Colab

Execute the next cell to install required libraries.

In [ ]:
colab_requirements = [
    "gymnasium",
    "numpy",
    "pandas",
    "seaborn"
]
import sys, subprocess
def run_subprocess_command(cmd):
    # run the command
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    # print the output
    for line in process.stdout:
        print(line.decode().strip())
        
if "google.colab" in sys.modules:
    for i in colab_requirements:
        run_subprocess_command("pip install " + i)

### If you use MyBinder

Required libraries are already installed, you have nothing to do.

### If you have downloaded the notebook on your computer and execute it in your own Python environment

Uncomment and execute the following cell to install required packages in your local environment (remove only the `#` not the `!`).

In [2]:
#!pip install gymnasium imageio numpy pandas seaborn

## Import required libraries

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import math
import gymnasium as gym
import numpy as np
import copy
import pandas as pd
import seaborn as sns

In [ ]:
sns.set_context("talk")

In [ ]:
#matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)

## Setup the FrozenLake toy problem with Gymnasium

For the purpose of focusing on the algorithms, we will use standard environments provided
by the Gymnasium framework. Especially, as in Lab 4, we will try to solve the FrozenLake-v1
problem (https://gymnasium.farama.org/environments/toy_text/frozen_lake/).
As a reminder, this environment is described [here](https://gymnasium.farama.org/environments/toy_text/frozen_lake/).

**Notice**: this environment is *fully observable*, thus here the terms (environment) *state* and (agent) *observation* are equivalent.
This is not always the case for example in poker, the agent doesn't know the opponent's cards.

### Display functions

The next cells contain two functions that can be used to display states and Qtables in the FrozenLake environment.

In [ ]:
def qtable_display(q_array, title=None, figsize=(4,4), annot=True, fmt="0.1f", linewidths=.5, square=True, cbar=False, cmap="Reds"):
    num_actions = q_array.shape[1]

    global_figsize = list(figsize)
    global_figsize[0] *= num_actions
    fig, ax_list = plt.subplots(ncols=num_actions, figsize=global_figsize)   # Sample figsize in inches

    for action_index in range(num_actions):
        ax = ax_list[action_index]
        state_seq = q_array.T[action_index]
        states_display(state_seq, title=None, figsize=figsize, annot=True, fmt="0.1f", linewidths=.5, square=True, cbar=False, cmap="Reds", ax=ax)

    plt.suptitle(title)
    plt.show()

In [ ]:
def states_display(state_seq, title=None, figsize=(5,5), annot=True, fmt="0.1f", linewidths=.5, square=True, cbar=False, cmap="Reds", ax=None):
    size = int(math.sqrt(len(state_seq)))
    state_array = np.array(state_seq)
    state_array = state_array.reshape(size, size)

    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)         # Sample figsize in inches

    sns.heatmap(state_array, annot=annot, fmt=fmt, linewidths=linewidths, square=square, cbar=cbar, cmap=cmap, ax=ax)
    
    if title is not None:
        plt.title(title)

    if ax is None:
        plt.show()
    else:
        return ax

## Exercise 1: Implement the TD Learning algorithm

In Lab4, we have seen Dynamic Programming methods that can be used to solve Markov Decision Problems when the environment is perfectly known to the agent i.e. in cases where the agent knows in advance the transition and the reward functions. This is a strong assumption as in most practical problems, these functions are not known in advance. In this lab, we will see how to make agents that can solve Markov Decision Problems without prior knowledge on the environment i.e. agents that learn the dynamics of their environment by exploring it and use this knowledge to find an optimal policy.

We will start with the *TD Learning* (*Temporal Difference Learning*) algorithm that can be used to **evaluate** any **given policy** (i.e. compute the *value function* or *V Table* of the environment following the given policy that is to say the expected value of any state when the agent follow the provided policy).
However, the main ideas of this algorithm can in turn be used within optimal control methods like e.g. https://en.wikipedia.org/wiki/TD-Gammon.
Exercises 2 to 4 also reuse the main concepts of TD Learning to calculate an optimal policy. 

The algorithm is recalled below.

---
TD Learning
---

<b>Input</b>:<br>
	$\quad\quad$ the policy $\pi$ to be evaluated<br>
<b>Algorithm parameter</b>:<br>
	$\quad\quad$ step size $\alpha \in (0,1]$<br><br>

Initialize arbitrarily $V(\boldsymbol{s}) ~~~ \forall \boldsymbol{s} \in \mathcal{S}$<br>
$V(\boldsymbol{s}_F) \leftarrow 0 ~~~ \forall \boldsymbol{s}_F \in \mathcal{S}^F$ (initialize finale states)<br><br>

<b>FOR EACH</b> episode<br>
	$\quad$ $S \leftarrow \text{env.reset}()$<br>
	$\quad$ <b>DO</b> <br>
		$\quad\quad$ $A \leftarrow \pi(S)$<br>
		$\quad\quad$ $S', R \leftarrow \text{env.step}(A)$<br>
		$\quad\quad$ $V(S) \leftarrow V(S) + \alpha \left[ \underbrace{\overbrace{R + \gamma ~ V(S')}^{\text{Target for } V(S)} ~ - ~ V(S)}_{\text{TD error}} \right]$<br>
		$\quad\quad$ $S \leftarrow S'$<br>
	$\quad$ <b>UNTIL</b> $S$ is final

**Notice**: in the following cell, `policy` is a list of actions (one per state c.f. two cells bellow).

In [ ]:
value_function_history = []
alpha_history = []

DISPLAY_EVERY_N_EPISODES = 50

def td_learning(policy, environment, alpha=0.1, alpha_factor=0.995, gamma=0.95, num_episodes=1000, display=False):
    num_states = environment.observation_space.n
    v_array = np.zeros(num_states)   # Initial value function

    for episode_index in range(num_episodes):
        if display and episode_index % DISPLAY_EVERY_N_EPISODES == 0:
            states_display(v_array, title="Value function (ep. {})".format(episode_index), cbar=True, cmap="Reds")
        else:
            print('.', end="")
        value_function_history.append(v_array.copy())
        alpha_history.append(alpha)
                
        if alpha_factor is not None:
            alpha = alpha * alpha_factor

        # TODO...
    
    return v_array

**Note**: In the following cell, the `display` argument can be set to `True` to see the evolution of the value function `v_array` over iterations.

In [ ]:
policy = [0, 3, 3, 3,
          0, 0, 0, 0,
          3, 1, 0, 0,
          0, 2, 1, 0]

environment = gym.make('FrozenLake-v1')
environment._max_episode_steps = 1000

v_array = td_learning(policy, environment, display=False)

environment.close()

states_display(v_array, title="Value function", cbar=True, cmap="Reds")

### Display the evolution of the value function over iterations

In [ ]:
df_v_hist = pd.DataFrame(value_function_history)
df_v_hist

Evolution of `v_array` (the estimated value of each state) over iterations (one curve per state):

In [ ]:
df_v_hist.plot(figsize=(14,8))
plt.title("V(s) w.r.t iteration")
plt.ylabel("V(s)")
plt.xlabel("iteration")
plt.legend(loc='upper right');

In [ ]:
plt.loglog(alpha_history)
plt.title("Alpha w.r.t iteration")
plt.ylabel("Alpha")
plt.xlabel("iteration");

## Exercise 2: The learning rate $\alpha$ in TD-Learning

In the previous exercise, set `alpha_factor` to 1 and check the algorithm with different values between 0 and 1.

What do you observe ?
What is the role of `alpha_factor` ?

## Exercise 3: Implement the Greedy and Epsilon Greedy policies

In exercise 1, TD-Learning has been used to **estimate the value** of a **given policy**.
In the following exercises, we will now see how to **find the optimal** (or a nearly optimal) policy.
For that, we will use two algorithms (SARSA and QLearning) that estimate a *QTable* (or *action-value function*) instead of a VTable (or value function).
This QTable gives the expected reward when the agent plays a given action $A$ from any given state $S$ and then follow a given *exploration policy* to choose the following actions until a final state is reached (and this exploration policy uses the QTable to choose actions to play).
While the agent explore the environment, it update it's QTable using an *update policy*.

The purpose of this third exercise is to implement the *greedy* and the $\epsilon$-*greedy* policies that agents will used to explore the environment and update their QTable:

$\displaystyle \pi^{Q^{\pi}}(S) := \text{greedy}(S, Q^{\pi}) = \arg\max_{A \in \mathcal{A}} Q^{\pi}(S, A)$


$\pi^{Q^{\pi},\epsilon}(S) := \epsilon\text{-greedy}(S, Q^{\pi}) = $
randomly choose between $\underbrace{\text{greedy}(S, Q^{\pi})}_{\text{with probability } 1 - \epsilon}$
and $~~ \underbrace{\text{a random action}}_{\text{with probability } \epsilon}$    i.e. $\epsilon \in (0,1]$

In [ ]:
def greedy_policy(state, q_array):

    # TODO...

    return action


def epsilon_greedy_policy(state, q_array, epsilon):

    # TODO...

    return action

## Exercise 4: Implement the SARSA algorithm

To find the optimal policy (or a nearly optimal policy) for the FrozenLake-v1 problem, we will first use the SARSA algorithm.
It is based on the online update of the so-called *QTable* (or *Q-function* or *action value function*) for the current policy defined as:
$$
Q^{\pi}(s, a) = \mathbb{E}^{\pi} \left[ \sum_{t=0}^{H} \gamma^t r(s_t, a_t) | s=s_0, a=a_0 \right] ,
$$
where $\gamma \in [0, 1]$ is the discount factor, and $H$ the horizon of the episode.

The SARSA algorithm updates a tabular estimate of the Q-function using the following update rule:
$$
Q^{\pi}_{t+1} (s_t , a_t) \leftarrow Q^{\pi}_t(s_t, a_t) + \alpha \left( r_t + \gamma Q^{\pi}_t(s_{t+1}, a_{t+1}) - Q^{\pi}_t(s_t, a_t) \right) ,
$$
where $\alpha \in (0, 1]$ is the learning rate, and $r_t$ is the reward received by the agent at time step $t$.
Most of the time, the SARSA algorithm is implemented with an $\epsilon$-greedy exploration strategy.
This strategy consists in selecting the best action learned so far with probability $(1 - \epsilon)$ and to select a random
1action with probability $\epsilon$.

**Tasks**: Implement the SARSA algorithm with $\epsilon$-greedy exploration (start with $\epsilon = 0.5$).

---
SARSA
---

<b>Input</b>:<br>
	$\quad\quad$ none<br>
<b>Algorithm parameter</b>:<br>
	$\quad\quad$ discount factor $\gamma$<br>
	$\quad\quad$ step size $\alpha \in (0,1]$<br>
	$\quad\quad$ small $\epsilon > 0$<br><br>

Initialize arbitrarily $Q(\boldsymbol{s}, \boldsymbol{a}) ~~~ \forall \boldsymbol{s} \in \mathcal{S}, \boldsymbol{a} \in \mathcal{A}(\boldsymbol{s})$<br>
$Q(\boldsymbol{s}_F, \cdot) = 0 ~~~ \forall \boldsymbol{s}_F \in \mathcal{S}^F$ (initialize finale states)<br><br>

<b>FOR EACH</b> episode<br>
	$\quad$ $S \leftarrow \text{env.reset}()$<br>
	$\quad$ $A \leftarrow \epsilon\text{-greedy}(S, Q)$<br>
	$\quad$ <b>DO</b> <br>
		$\quad\quad$ $R, S' \leftarrow \text{env.step}(A)$<br>
		$\quad\quad$ $A' \leftarrow \epsilon\text{-greedy}(S', Q)$<br>
		$\quad\quad$ $Q(S,A) \leftarrow Q(S,A) + \alpha \left[ \underbrace{R + \gamma ~ Q(S',A') ~ - ~ Q(S,A)}_{\text{TD error}} \right]$<br>
		$\quad\quad$ $S \leftarrow S'$<br>
		$\quad\quad$ $A \leftarrow A'$<br>
	$\quad$ <b>UNTIL</b> $S$ is final

In [ ]:
q_array_history = []
alpha_history = []

DISPLAY_EVERY_N_EPISODES = 50

def sarsa(environment, alpha=0.1, alpha_factor=0.9995, gamma=0.99, epsilon=0.5, num_episodes=10000, display=False):
    num_states = environment.observation_space.n
    num_actions = environment.action_space.n
    q_array = np.zeros([num_states, num_actions])   # Initial Q table

    for episode_index in range(num_episodes):
        if display and episode_index % DISPLAY_EVERY_N_EPISODES == 0:
            qtable_display(q_array, title="Q table", cbar=True)
        else:
            print('.', end="")
        q_array_history.append(q_array.copy())
        alpha_history.append(alpha)

        # Update alpha
        if alpha_factor is not None:
            alpha = alpha * alpha_factor
        
        # TODO...

    return q_array

**Note**: In the following cell, the `display` argument can be set to `True` to see the evolution of the action-value function `q_array` over iterations.

In [ ]:
environment = gym.make('FrozenLake-v1')
environment._max_episode_steps = 1000

q_array = sarsa(environment, display=False)

environment.close()

qtable_display(q_array, title="Q Table", cbar=True)

### Display the evolution of the value function over iterations

In [ ]:
q_array_history = np.array(q_array_history)
df_q_hist_list = []

for action_index in range(q_array_history.shape[2]):
    df_q_hist_list.append(pd.DataFrame(q_array_history[:, :, action_index]))

Evolution of `q_array` over iterations (one curve per state):

In [ ]:
for action_index, df_q_hist in enumerate(df_q_hist_list):
    df_q_hist.plot(figsize=(14,8))
    plt.title("Q(s,{}) w.r.t iteration".format(action_index))
    plt.ylabel("Q(s,{})".format(action_index))
    plt.xlabel("iteration")
    plt.legend(loc='upper right');

In [ ]:
plt.loglog(alpha_history)
plt.title("Alpha w.r.t iteration")
plt.ylabel("Alpha")
plt.xlabel("iteration");

### Evaluate Policy with Gymnasium

As a measure of performance, count the number of successful trials on 1000 episodes.

**Note**: Gymnasium considers the task is solved if you reach 76\% of success over the episodes.

In [ ]:
environment = gym.make('FrozenLake-v1')
environment._max_episode_steps = 1000

reward_list = []
NUM_EPISODES = 1000

for episode_index in range(NUM_EPISODES):
    state, info = environment.reset()
    done = False
    #t = 0

    while not done:
        #action = epsilon_greedy_policy(state, q_array, epsilon)
        action = greedy_policy(state, q_array)
        state, reward, done, truncated, info = environment.step(action)

    reward_list.append(reward)

reward_df = pd.DataFrame(reward_list)

print('Average reward (which is equivalent to a "success rate" in the FrozenLake environment as the total rewards in this environment are either 0 or 1):', np.average(reward_df))

environment.close()

## Exercise 5: Implement the QLearning algorithm

Another reinforcement learning algorithm is the so called Q-Learning algorithm.
The fundamental difference with SARSA is that it is an off-policy algorithm.
This means that it doesn't estimate the Q-function of its current policy but it estimates the value of another policy which is the optimal one.
To do so, it uses the following update rule:
$$
Q_{t+1}(s_t, a_t) \leftarrow Q_t(s_t, a_t) + \alpha \left( r_t + \gamma \max_b Q_t(s_{t+1}, b) - Q_t(s_t, a_t) \right) .
$$

**Task**: in this exercise, you will replace the SARSA update rule by the Q-learning one and analyze the
differences in performances.

---
QLearning
---

<b>Input</b>:<br>
	$\quad\quad$ none<br>
<b>Algorithm parameter</b>:<br>
	$\quad\quad$ discount factor $\gamma$<br>
	$\quad\quad$ step size $\alpha \in (0,1]$<br>
	$\quad\quad$ small $\epsilon > 0$<br><br>

Initialize arbitrarily $Q(\boldsymbol{s}, \boldsymbol{a}) ~~~ \forall \boldsymbol{s} \in \mathcal{S}, \boldsymbol{a} \in \mathcal{A}(\boldsymbol{s})$<br>
$Q(\boldsymbol{s}_F, \cdot) = 0 ~~~ \forall \boldsymbol{s}_F \in \mathcal{S}^F$ (initialize finale states)<br><br>

<b>FOR EACH</b> episode<br>
	$\quad$ $S \leftarrow \text{env.reset}()$<br>
	$\quad$ <b>DO</b> <br>
		$\quad\quad$ $A \leftarrow \epsilon\text{-greedy}(S, Q)$<br>
		$\quad\quad$ $R, S' \leftarrow \text{env.step}(A)$<br>
		$\quad\quad$ $Q(S,A) \leftarrow Q(S,A) + \alpha \left[ \underbrace{R + \gamma ~ \max_{\boldsymbol{a}} Q(S', \boldsymbol{a}) ~ - ~ Q(S,A)}_{\text{TD error}} \right]$<br>
		$\quad\quad$ $S \leftarrow S'$<br>
	$\quad$ <b>UNTIL</b> $S$ is final

In [ ]:
q_array_history = []
alpha_history = []

DISPLAY_EVERY_N_EPISODES = 50

def q_learning(environment, alpha=0.1, alpha_factor=0.9995, gamma=0.99, epsilon=0.5, num_episodes=10000, display=False):
    num_states = environment.observation_space.n
    num_actions = environment.action_space.n
    q_array = np.zeros([num_states, num_actions])   # Initial Q table

    for episode_index in range(num_episodes):
        if display and episode_index % DISPLAY_EVERY_N_EPISODES == 0:
            qtable_display(q_array, title="Q table", cbar=True)
        else:
            print('.', end="")
        q_array_history.append(q_array.copy())
        alpha_history.append(alpha)

        # Update alpha
        if alpha_factor is not None:
            alpha = alpha * alpha_factor

        # TODO...
    
    return q_array

**Note**: In the following cell, the `display` argument can be set to `True` to see the evolution of the action-value function `q_array` over iterations.

In [ ]:
environment = gym.make('FrozenLake-v1')
environment._max_episode_steps = 1000

q_array = q_learning(environment, display=False)

environment.close()

qtable_display(q_array, title="Q Table", cbar=True)

### Display the evolution of the value function over iterations

In [ ]:
q_array_history = np.array(q_array_history)
df_q_hist_list = []

for action_index in range(q_array_history.shape[2]):
    df_q_hist_list.append(pd.DataFrame(q_array_history[:, :, action_index]))

Evolution of `q_array` over iterations (one curve per state):

In [ ]:
for action_index, df_q_hist in enumerate(df_q_hist_list):
    df_q_hist.plot(figsize=(14,8))
    plt.title("Q(s,{}) w.r.t iteration".format(action_index))
    plt.ylabel("Q(s,{})".format(action_index))
    plt.xlabel("iteration")
    plt.legend(loc='upper right');

In [ ]:
plt.loglog(alpha_history)
plt.title("Alpha w.r.t iteration")
plt.ylabel("Alpha")
plt.xlabel("iteration");

### Evaluate Policy with Gymnasium

As a measure of performance, count the number of successful trials on 1000 episodes.

**Note**: Gymnasium considers the task is solved if you reach 76\% of success over the episodes.

In [ ]:
environment = gym.make('FrozenLake-v1')
environment._max_episode_steps = 1000

reward_list = []
NUM_EPISODES = 1000

for episode_index in range(NUM_EPISODES):
    state, info = environment.reset()
    done = False
    #t = 0

    while not done:
        #action = epsilon_greedy_policy(state, q_array, epsilon)
        action = greedy_policy(state, q_array)
        state, reward, done, truncated, info = environment.step(action)

    reward_list.append(reward)

reward_df = pd.DataFrame(reward_list)

print('Average reward (which is equivalent to a "success rate" in the FrozenLake environment as the total rewards in this environment are either 0 or 1):', np.average(reward_df))

environment.close()